In [3]:
import keras
from tensorflow.keras.datasets import cifar100, fashion_mnist,mnist, cifar10
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers,callbacks
#GPU
import os
import gc
import tensorflow as tf
gc.collect()

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "2"
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))

In [ ]:
keras.__version__, tf.__version__

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [6]:
y_train=tf.keras.utils.to_categorical(y_train, 10)
y_test=tf.keras.utils.to_categorical(y_test,10)

In [7]:
import numpy as np
import random
def randomCrop(img_org):
    width=32
    height=32
    img = np.zeros((32+8,+32+8,3))
    img[4::,4:]=img_org
    assert img.shape[0] >= height
    assert img.shape[1] >= width
    x = random.randint(0, img.shape[1] - width)
    y = random.randint(0, img.shape[0] - height)
    img = img[y:y+height, x:x+width]
    return img
datagen = ImageDataGenerator(
    preprocessing_function=randomCrop, horizontal_flip=True)
datagen.fit(X_train)

In [8]:
# Use scikit-learn to grid search the number of neurons

# Function to create model, required for KerasClassifier
def create_model(depth=3,length=2, junction=3,initial_filter=8,ApplyTransformer=False,type_of_block="inception"):
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"]= "2"
    from tensorflow.compat.v1.keras.backend import set_session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.compat.v1.Session(config=config))


    import numpy
    from plexusnet.architecture import PlexusNet,Configuration
    import tensorflow_addons as tfa
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import Dropout
    from keras.constraints import maxnorm
    # create model
    Configuration["num_heads"]=4
    model=PlexusNet(depth=depth,length=length,initial_filter=initial_filter,junction=junction,n_class=10, input_shape=(32,32), type_of_block=type_of_block,ApplyTransformer=ApplyTransformer,ApplyLayerNormalization=True, run_all_BN=False,GlobalPooling="avg").model
    opt=tf.optimizers.Adam()
    model.compile(optimizer=opt, metrics=["acc"], loss="categorical_crossentropy")   
    return model

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model, epochs=1, batch_size=64, verbose=1)

depth = [3]
length= [3,4,5,6,7,8]
junction= [1,2,3]
ApplyTransformer=[False,True]
initial_filter=[4,8,16,24]
type_of_block=["inception", "resnet", "vgg", "soft_att"]
param_grid = dict(depth=depth,
                  length=length,
                  junction=junction,
                  ApplyTransformer=ApplyTransformer,
                type_of_block=type_of_block,
                  initial_filter=initial_filter
                 )
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=8, cv=2,verbose=100)
grid_result = grid.fit(X_train, y_train)

In [ ]:
grid_result

In [18]:
import pandas as pd
pd.DataFrame(grid_result.cv_results_).sort_values("mean_test_score",ascending=False).to_csv("./search_result_plexusnet.csv")

In [ ]:
grid_result.best_params_

In [ ]:
grid_result.best_score_

In [ ]:
grid_result.best_estimator_